# Improved Implementation for Stochastic Linear Regression
with regularization.\
with efficiency improvements.\
Recommended to check cost after convergence.

In [1]:
import numpy as np

In [2]:
from scripts import LinearRegression

In [3]:
?LinearRegression.fit

Signature:
LinearRegression.fit(
    self,
    X,
    y,
    epochs=100,
    alpha=0.01,
    Lambda=0.0001,
    error_threshold=0.001,
    validation_size=0.2,
    output_limit=10,
)
Docstring:
Fit the linear regression model to the given data.

Parameter
---------
epochs: int, default=1000
    Number of complete iterations through X

alpha : float, default=0.01
    Constant Learning Rate

Lambda : float, default=0.0001
    Rate for l2 Regularization

error_threshold: float, default=0.001
    Threshold for vCost convergence

validation_size: float, default=0.2
    Percent of data for validation, 0 <= vs < 1

output_limit : int, default=10
    Number of iterations to show

Returns
-------
W : numpy.ndarray
    The optimized weights.
b : numpy.longdouble
    The optimized intercept.
File:      ~/Desktop/ml-algorithms/notebooks/scripts/linear_regression.py
Type:      function

## Usage

In [4]:
m = LinearRegression()
X = np.random.rand(1000,2)
y = 5.55*X[:,0] + 11.22*X[:,1] + 50
m.fit(X, y ,epochs= 1000, alpha = 0.2, error_threshold = 0.00001, output_limit=10)

(0/10) > Epoch: 0 cost: 3284.17263479 vCost: 3291.50451516
(1/10) > Epoch: 100 cost: 0.53181002 vCost: 0.63992793
(2/10) > Epoch: 200 cost: 0.02491143 vCost: 0.02997905
(3/10) > Epoch: 300 cost: 0.00138049 vCost: 0.00165955
       * Epoch: 363 vCost: 0.00034998

Epoch 363 > vCost Converged with threshold 1e-05. Or performance degraded.


(array([ 5.5267, 11.1665]), 50.034201252729396)

In [5]:
from sklearn.datasets import make_regression

In [6]:
X, y = make_regression(n_samples=1000,n_features=20, n_informative=19)
m = LinearRegression()
m.fit(X, y ,epochs= 100, alpha = 0.5, error_threshold = 0.01, output_limit=10)

(0/10) > Epoch: 0 cost: 54843.40469729 vCost: 61494.34831012
(1/10) > Epoch: 10 cost: 9.19630793 vCost: 9.26865817
(2/10) > Epoch: 20 cost: 0.00842255 vCost: 0.00841395
       * Epoch: 28 vCost: 0.00442379

Epoch 28 > vCost Converged with threshold 0.01. Or performance degraded.


(array([6.9366e+01, 6.3152e-04, 6.1716e+01, 6.7187e+01, 2.8868e+01,
        8.4267e+01, 7.5727e+01, 1.7868e+01, 6.1031e+01, 3.5947e+00,
        9.4115e+01, 4.1595e+01, 8.8472e+01, 1.7439e+01, 3.9721e+00,
        3.3800e+00, 7.0854e+01, 3.0207e+01, 2.0141e+01, 2.9645e+01]),
 -0.0157228756082394)

In [16]:
from sklearn.datasets import fetch_california_housing

In [26]:
california = fetch_california_housing(as_frame=True)

In [42]:
X,y = california["data"], california["target"]
X,y = X.to_numpy(), y.to_numpy()

In [38]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [43]:
m = LinearRegression()
m.fit(X, y ,epochs= 25, alpha = 0.0001, error_threshold = 1/1000, validation_size=1/3 ,output_limit=10)

(0/10) > Epoch: 0 cost: 27655.30556463 vCost: 25204.06381210
(1/10) > Epoch: 2 cost: 60.39310731 vCost: 64.45519541
(2/10) > Epoch: 4 cost: 55.16757021 vCost: 57.38310832
(3/10) > Epoch: 6 cost: 53.15122585 vCost: 55.03569282
(4/10) > Epoch: 8 cost: 51.26560491 vCost: 53.03803963
(5/10) > Epoch: 10 cost: 49.45786300 vCost: 51.14868625
(6/10) > Epoch: 12 cost: 47.72518217 vCost: 49.33891925
(7/10) > Epoch: 14 cost: 46.06465769 vCost: 47.60234180
(8/10) > Epoch: 16 cost: 44.47334082 vCost: 45.93556844
(9/10) > Epoch: 18 cost: 42.94837778 vCost: 44.33572240
(10/10) > Epoch: 20 cost: 41.48702775 vCost: 42.80009359
(11/10) > Epoch: 22 cost: 40.08666146 vCost: 41.32608826
(12/10) > Epoch: 24 cost: 38.74475708 vCost: 39.91121829


(array([0.8619, 0.4953, 0.6694, 0.987 , 0.0027, 0.0409, 1.0096, 0.5796]),
 10.683817879341833)

In [44]:
?np.clip

Signature:       np.clip(a, a_min, a_max, out=None, **kwargs)
Call signature:  np.clip(*args, **kwargs)
Type:            _ArrayFunctionDispatcher
String form:     <function clip at 0x7e9ec00537e0>
File:            ~/miniconda3/envs/ml-algorithms/lib/python3.12/site-packages/numpy/core/fromnumeric.py
Docstring:      
Clip (limit) the values in an array.

Given an interval, values outside the interval are clipped to
the interval edges.  For example, if an interval of ``[0, 1]``
is specified, values smaller than 0 become 0, and values larger
than 1 become 1.

Equivalent to but faster than ``np.minimum(a_max, np.maximum(a, a_min))``.

No check is performed to ensure ``a_min < a_max``.

Parameters
----------
a : array_like
    Array containing elements to clip.
a_min, a_max : array_like or None
    Minimum and maximum value. If ``None``, clipping is not performed on
    the corresponding edge. Only one of `a_min` and `a_max` may be
    ``None``. Both are broadcast against `a`.
out : ndarray